---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [39]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [40]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [41]:
def graph_identification():
    
    print(nx.info(P1_Graphs[0]))
    print("")
    print(nx.info(P1_Graphs[1]))
    print("")
    print(nx.info(P1_Graphs[2]))
    print("")
    print(nx.info(P1_Graphs[3]))
    print("")
    print(nx.info(P1_Graphs[4]))
    return ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

graph_identification()



Name: barabasi_albert_graph(1000,2)
Type: Graph
Number of nodes: 1000
Number of edges: 1996
Average degree:   3.9920

Name: watts_strogatz_graph(1000,10,0.05)
Type: Graph
Number of nodes: 1000
Number of edges: 5000
Average degree:  10.0000

Name: watts_strogatz_graph(750,5,0.075)
Type: Graph
Number of nodes: 750
Number of edges: 1500
Average degree:   4.0000

Name: barabasi_albert_graph(750,4)
Type: Graph
Number of nodes: 750
Number of edges: 2984
Average degree:   7.9573

Name: watts_strogatz_graph(750,4,1)
Type: Graph
Number of nodes: 750
Number of edges: 1500
Average degree:   4.0000


['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [42]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [66]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

def has_management_salary(node):
    managementSalary = node[1]['ManagementSalary']
    if managementSalary == 0:
        return 0
    elif managementSalary == 1:
        return 1
    else:
        return None

def salary_predictions():
    
    df = pd.DataFrame(index=G.nodes())
    
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['centrality'] = pd.Series(nx.degree_centrality(G))
    df['closeness'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    df['betweeness'] = pd.Series(nx.betweenness_centrality(G, normalized=True))
    df['page_rank'] = pd.Series(nx.pagerank(G))
    
    df['has_management_salary'] = pd.Series([has_management_salary(node) for node in G.nodes(data=True)])
    
    # Create the dataframe of train and test
    df_train = df[~pd.isnull(df['has_management_salary'])]
    df_test = df[pd.isnull(df['has_management_salary'])]
    
    # Define the features
    features = ['clustering', 'degree', 'centrality', 'closeness', 'betweeness', 'page_rank']
    X_train = df_train[features]
    X_test = df_test[features]
    
    # Create the series with result
    y_train = df_train['has_management_salary']
    #y_test = df_test['has_management_salary']
    
    # Define the classifier
    classifier = RandomForestClassifier(n_estimators = 30, random_state=1).fit(X_train, y_train)
    
    print('Accuracy of RF classifier on training set: {:.2f}'.format(classifier.score(X_train, y_train)))
    #print('Accuracy of RF classifier on test set: {:.2f}'.format(classifier.score(X_test, y_test)))
    
    df_predict = pd.DataFrame(classifier.predict_proba(X_test), columns=["a","b"])
    df_predict['b'] = df_predict['b'].astype('float32', casting='same_kind')
     
    series = pd.Series(df_predict['b']) 
    series.index = df_test.index
    
    return series

salary_predictions()

Accuracy of RF classifier on training set: 1.00


1       0.066667
2       0.966667
5       1.000000
8       0.300000
14      0.033333
18      0.100000
27      0.066667
30      0.733333
31      0.366667
34      0.066667
37      0.066667
40      0.066667
45      0.000000
54      0.400000
55      0.600000
60      0.300000
62      1.000000
65      1.000000
77      0.033333
79      0.066667
97      0.000000
101     0.000000
103     0.433333
108     0.000000
113     0.100000
122     0.000000
141     0.133333
142     1.000000
144     0.000000
145     0.566667
          ...   
913     0.000000
914     0.000000
915     0.000000
918     0.166667
923     0.000000
926     0.066667
931     0.000000
934     0.000000
939     0.000000
944     0.000000
945     0.000000
947     0.000000
950     0.033333
951     0.000000
953     0.000000
959     0.000000
962     0.000000
963     0.166667
968     0.000000
969     0.000000
974     0.000000
984     0.033333
987     0.000000
989     0.000000
991     0.000000
992     0.000000
994     0.000000
996     0.0000

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [67]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [71]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing


def new_connections_predictions():
    
    # Computing of preferential attachment score
    series_preferential_attachment = nx.preferential_attachment(G, future_connections.index)
    
    # Jaccard coefficient measures similarity between finite sample sets
    series_jaccard_coefficient = nx.jaccard_coefficient(G, future_connections.index)
    
    # Similarity of endpoints depends on the potential resource interacted between them
    series_resource_allocation_index = nx.resource_allocation_index(G, future_connections.index)
    
    # Predict links in a network usind the number of links shared between 2 nodes
    series_adamic_adar_index = nx.adamic_adar_index(G, future_connections.index)
    
    # Number of common neighbors and bonus one for each common neighbor belonging to the same community
    series_cn_soundarajan_hopcroft = nx.cn_soundarajan_hopcroft(G, future_connections.index)
    
    # Number of resource allocation index considering only common neighbors belonging to the same community
    series_ra_index_soundarajan_hopcroft = nx.ra_index_soundarajan_hopcroft(G, future_connections.index)
    
    nodes_disconected = future_connections.index
    
    future_connections['preferencial_attachment'] = [i[2] for i in series_preferential_attachment]
    future_connections['common_neighbors'] = [len(list(nx.common_neighbors(G, node[0], node[1]))) for node in nodes_disconected]
    future_connections['jaccard_coefficient'] = [i[2] for i in series_jaccard_coefficient]
    future_connections['resource_allocation'] = [i[2] for i in series_resource_allocation_index]
    future_connections['adamic_adar'] = [i[2] for i in series_adamic_adar_index]
    
    for node in G.nodes():
        G.node[node]['community'] = G.node[node]['Department']
    future_connections['cn_soundarajan_hopcroft'] = [i[2] for i in series_cn_soundarajan_hopcroft]   
    future_connections['ra_soundarajan_hopcroft'] = [i[2] for i in series_ra_index_soundarajan_hopcroft]
    
    # Create the dataframe of train and test
    df_train = future_connections[~pd.isnull(future_connections['Future Connection'])]
    df_test = future_connections[pd.isnull(future_connections['Future Connection'])]
    
    # Define the features
    features = ['preferencial_attachment', 'common_neighbors','jaccard_coefficient', 'resource_allocation', 
                'adamic_adar','cn_soundarajan_hopcroft', 'ra_soundarajan_hopcroft']
    
    X_train = df_train[features]
    X_test = df_test[features]
    
    # Create the series with result
    y_train = df_train['Future Connection']
    
    # Define the classifier
    classifier = RandomForestClassifier(n_estimators = 30, random_state=1).fit(X_train, y_train)
    print('Accuracy of RF classifier on training set: {:.2f}'.format(classifier.score(X_train, y_train)))
    
    df_predict = pd.DataFrame(classifier.predict_proba(X_test), columns=["a","b"])
    df_predict['b'] = df_predict['b'].astype('float32', casting='same_kind')
     
    series = pd.Series(df_predict['b']) 
    series.index = df_test.index
    
    return series

new_connections_predictions()

Accuracy of RF classifier on training set: 0.99


(107, 348)    0.000000
(542, 751)    0.012920
(20, 426)     0.466667
(50, 989)     0.013654
(942, 986)    0.010818
(324, 857)    0.016408
(13, 710)     0.066667
(19, 271)     0.000000
(319, 878)    0.012811
(659, 707)    0.015444
(49, 843)     0.014750
(208, 893)    0.005232
(377, 469)    0.000000
(405, 999)    0.022222
(129, 740)    0.000000
(292, 618)    0.000000
(239, 689)    0.006106
(359, 373)    0.170094
(53, 523)     0.300000
(276, 984)    0.006901
(202, 997)    0.009603
(604, 619)    0.066667
(270, 911)    0.014154
(261, 481)    0.000000
(200, 450)    1.000000
(213, 634)    0.017382
(644, 735)    0.000000
(346, 553)    0.000000
(521, 738)    0.018560
(422, 953)    0.000000
                ...   
(672, 848)    0.014154
(28, 127)     1.000000
(202, 661)    0.009012
(54, 195)     1.000000
(295, 864)    0.015147
(814, 936)    0.011965
(839, 874)    0.010818
(139, 843)    0.013573
(461, 544)    0.015761
(68, 487)     0.000000
(622, 932)    0.016030
(504, 936)    0.000000
(479, 528) 